In [ ]:
# Install Great Expectations if not already installed
# !pip install great_expectations

import pandas as pd
import great_expectations as gx
from great_expectations.checkpoint import SimpleCheckpoint

# Load a sample dataset
df = pd.read_csv("https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv")

# Display the first few rows
print("Sample Data:")
print(df.head())

# Initialize an Ephemeral Data Context (in-memory, no files written to disk)
context = gx.get_context()

# Create a Datasource and Data Asset
datasource = context.sources.add_pandas(name="my_pandas_datasource")
data_asset = datasource.add_dataframe_asset(name="titanic_asset", dataframe=df)

# Build a Batch Request
batch_request = data_asset.build_batch_request()

# Create an Expectation Suite
expectation_suite_name = "titanic_sla_expectations"
context.add_expectation_suite(expectation_suite_name=expectation_suite_name)

# Get a Validator
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# Define Data Quality SLAs as Expectations
# 1. Completeness: Age column should not have null values
validator.expect_column_values_to_not_be_null("Age")

# 2. Valid Range: Age should be between 0 and 100
validator.expect_column_values_to_be_between("Age", min_value=0, max_value=100)

# 3. Valid Categories: Sex should be either 'male' or 'female'
validator.expect_column_values_to_be_in_set("Sex", ["male", "female"])

# 4. Completeness: Fare column should not have null values
validator.expect_column_values_to_not_be_null("Fare")

# 5. Valid Range: Fare should be between 0 and 600
validator.expect_column_values_to_be_between("Fare", min_value=0, max_value=600)

# 6. Completeness: Embarked column should not have null values
validator.expect_column_values_to_not_be_null("Embarked")

# 7. Valid Categories: Embarked should be one of 'C', 'Q', or 'S'
validator.expect_column_values_to_be_in_set("Embarked", ["C", "Q", "S"])

# Save the Expectation Suite
validator.save_expectation_suite(discard_failed_expectations=False)

# Create a Simple Checkpoint
checkpoint = SimpleCheckpoint(
    name="titanic_sla_checkpoint",
    data_context=context,
    validator=validator
)

# Run the Checkpoint
checkpoint_result = checkpoint.run()

# Display the Validation Results
print("\nValidation Results:")
print(checkpoint_result)

# Optionally, display a summary of validation results
from great_expectations.render.renderer import ValidationResultsPageRenderer
from great_expectations.render.view import DefaultJinjaPageView
from IPython.core.display import display, HTML

document_model = ValidationResultsPageRenderer().render(checkpoint_result)
html = DefaultJinjaPageView().render(document_model)
display(HTML(html))